# 라이브러리 정의

In [48]:
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [49]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 분석 알고리즘 DecisionTree 구현 라이브러리
# 설명력은 좀 낮지만 정확도 높음
from sklearn.tree import DecisionTreeRegressor

# 과거데이터를 8:2, 7:3이나 이런식으로 자동으로 나누어주는 라이브러리
from sklearn.model_selection import train_test_split 

# 라벨 인코더 -> 문자를 숫자로 맵핑시켜준다. 두개는 0,1 세개는 0,1,2 네개는 0,1,2,3
from sklearn.preprocessing import LabelEncoder

In [50]:
 # 분석 Regression!!   대표 평가지표  (MAE, RMSE)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error  

In [7]:
# from sklearn.ensemble import RandomForestRegressor
# 설명력이 뛰어남

# from sklearn.linear_model import LinearRegression
# 연속적 예측

In [8]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


# 데이터 불러오기

In [9]:
featuresData = \
    pd.read_csv("../dataset/feature_regression_example.csv")

# 1.데이터 전처리

### 1-1. 타입통합/ 특성 숫자컬럼 추가

### 1-1-1. 데이터 타입 통합

In [10]:
# featuresData.info()

In [11]:
# 주의할 사항은 모든 컬럼에 대해서 고정시키는걸 고려하는게 나을 수도 있다
featuresData.QTY = featuresData.QTY.astype(float)  # qty타입을 플롯으로 바꾼다

### 1-1-2. 특성 값 숫자컬럼 변경

In [12]:
#  이유 : 머신러닝 특성은 숫자형 지원
# 해결 : Y->1, N->0 과 같이 변환

In [13]:
featuresData.columns

Index(['REGIONID', 'PRODUCTGROUP', 'PRODUCT', 'ITEM', 'YEARWEEK', 'YEAR',
       'WEEK', 'QTY', 'HOLIDAY', 'HCLUS', 'PROMOTION', 'PRO_PERCENT'],
      dtype='object')

In [14]:
featuresData.PROMOTION.drop_duplicates()

0    Y
9    N
Name: PROMOTION, dtype: object

In [15]:
featuresData["HOLIDAY_NEW"] = np.where(featuresData.HOLIDAY=="Y",1,0)

In [16]:
featuresData["PROMOTION_NEW"] = np.where(featuresData.PROMOTION=="Y",1,0)


In [17]:
# 이 모양으로도 가능
# pd.DataFrame( featuresData.HOLIDAY.drop_duplicates() ) .reset_index().rename(columns={"index" : "HOLIDAY_ENCODE"})

# 1-2. 특성 선정/ 데이터 분리(특성 데이터 추출)

### 1-2-1. 특성선정

In [18]:
corrDf = featuresData.corr() # corr() -> 상관계수를 뽑을 수 있다 yearweek와 qty의 상관성을 알 수 있다.

In [19]:
standardLimit = 0.5

In [20]:
# 반응하는 변수들 
corrDf.loc[ (abs(corrDf.QTY) > standardLimit) & (corrDf.QTY !=1)]

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,HOLIDAY_NEW,PROMOTION_NEW
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619,-0.974601,-0.374072
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000,0.487062,0.898554
HOLIDAY_NEW,0.049867,-0.031106,0.273710,0.505932,-0.974601,0.487062,1.000000,0.365148
PROMOTION_NEW,0.205916,0.195931,0.060206,0.612451,-0.374072,0.898554,0.365148,1.000000


In [21]:
# standard limit에 따라 features가 바뀔 수 있음
features = \
    list( corrDf.loc[ (abs(corrDf.QTY) > standardLimit) & 
                     (corrDf.QTY != 1) ].index )

In [22]:
features

['HCLUS', 'PRO_PERCENT', 'HOLIDAY_NEW', 'PROMOTION_NEW']

In [23]:
label = ["QTY"]

### 1-2-2. 데이터 분리

In [24]:
# 과거실적 데이터를 7:3, 8:2정도로 분리를 시킨다 (70퍼센트 정도를 컴퓨터에게 훈련 시키고 나머지 30퍼셑느로 답을 비교함)

In [25]:
standardIndex = 0.8

In [26]:
featuresData.shape

(105, 14)

In [27]:
sortKey = ["REGIONID","ITEM", "YEARWEEK"]

In [28]:
sortedData = featuresData.sort_values(sortKey, ignore_index=True)

In [29]:
selectedIndex = int( (sortedData.shape )[0] * 0.8 ) 
# 정렬하고 80%에 있는 인덱스의 번호로 분리시킨다~!

In [30]:
yearweekStd = sortedData.loc[selectedIndex].YEARWEEK
yearweekStd

201632

In [31]:
# 훈련데이터와 테스트데이터를 (문제지와 정답지로 구분해서 정의한다.)(문제, 정답, 문제, 정답)
trainingDataFeatures = \
    sortedData.loc[sortedData.YEARWEEK <= yearweekStd, features] # 과거 문제지
trainingDataLabel = \
    sortedData.loc[sortedData.YEARWEEK <= yearweekStd, label] # 과거 정답지
testDataFeatures = \
    sortedData.loc[sortedData.YEARWEEK > yearweekStd, features] # 최신 문제지
testDataLabel = \
    sortedData.loc[sortedData.YEARWEEK > yearweekStd, label] # 최신 정답지 
                                                             # 가장 최신 정답지(20,30%)는 항상 비교군이다 (이미 최신 정답지는 있음, 과거의 피쳐를 돌려서 얼마나 정확하게 예측하는지 비교하기 위해서 필요한 것)

In [32]:
trainingDataFeatures

,HCLUS,PRO_PERCENT,HOLIDAY_NEW,PROMOTION_NEW
0,1,0.209442,1,1
1,4,0.209442,0,1
2,4,0.208155,0,1
3,2,0.208155,1,1
4,4,0.208155,0,1
...,...,...,...,...
80,1,0.308584,1,1
81,1,0.308584,1,1
82,4,0.280258,0,1
83,4,0.280258,0,1


# 2. 모델적용

### 2-1. 모델적용

### 2-1-1. 학습

In [33]:
# 모델 선언(Decision, Random)
model_dt = DecisionTreeRegressor(random_state=10)
model_rf = RandomForestTreeRegressor(random_state=10)

NameError: name 'RandomForestTreeRegressor' is not defined

In [34]:
model_rf.fit(X=trainingDataFeatures, y=trainingDataLabel)  ## 다 못침ㅜ

NameError: name 'model_rf' is not defined

In [35]:
# 하이퍼 파라미터 튜닝!!..?..?ㅋㅋㅋ?ㅋㅋㅋ??ㅋ??
model = DecisionTreeRegressor(random_state=10)

# x는 문제지 y는 결과물 
model.fit(X = trainingDataFeatures, y=trainingDataLabel)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=10, splitter='best')

# 3. 예측

In [36]:
model_dt.ValueDt = model_dt.predict(testDataFeatures)

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [37]:
predictValueRf = model_rf.predict(testDataFeatures)

NameError: name 'model_rf' is not defined

In [38]:
predictDtDf = pd.DataFrame(predictBalueDt, columns=["PREDICT_DT"])
predictRfDf = pd.DataFrame(predictBalueDt, columns=["PREDICT_DT"])

NameError: name 'predictBalueDt' is not defined

In [39]:
# predictValue = model.predict(testDataFeatures)

In [186]:
# predictValue

array([ 350.71428571, 1434.6       , 1434.6       , 1434.6       ,
       1434.6       , 1434.6       , 1708.75      , 1708.75      ,
       2620.42857143, 1708.75      ,  350.71428571, 2193.8       ,
       2193.8       , 2193.8       , 2193.8       , 2193.8       ,
       1434.6       , 1434.6       , 1434.6       , 1434.6       ])

In [187]:
# predictDf = pd.DataFrame( list(predictValue), columns=["PREDICT"] )

# 4. 데이터 정리

In [ ]:
 # 3.예측 과 4. 데이터정리  를 데이터 전처리로 합친다

In [188]:
validateDf = \
    pd.concat( [testDataLabel.reset_index(drop=True), predictDf ], axis = 1)
validateDf

,QTY,PREDICT
0,43.0,350.714286
1,1700.0,1434.600000
2,1514.0,1434.600000
3,1501.0,1434.600000
4,1491.0,1434.600000
5,806.0,1434.600000
6,2111.0,1708.750000
7,2400.0,1708.750000
8,2010.0,2620.428571
9,1900.0,1708.750000


# 5. 정확도 검증

In [189]:
mae = mean_absolute_error( y_true=validateDf.QTY,   # mae???
                   y_pred=validateDf.PREDICT )

In [190]:
rmse = np.sqrt( mean_squared_error(y_true = validateDf.QTY, # rmse ???
                           y_pred=validateDf.PREDICT) )

In [191]:
validateDf

,QTY,PREDICT
0,43.0,350.714286
1,1700.0,1434.600000
2,1514.0,1434.600000
3,1501.0,1434.600000
4,1491.0,1434.600000
5,806.0,1434.600000
6,2111.0,1708.750000
7,2400.0,1708.750000
8,2010.0,2620.428571
9,1900.0,1708.750000


In [98]:
### 장점
### DecisionTree -> 과거의 겅험치를 그대로 반영한다~!
###                 변동성이 큰 데이터에서 강하다~!

### 단점
### 오버피팅 문제가 생김!! -> 너무 과거에 얽매여서 미래에대한 어떠한 포인트도 줄 수 없다. (과거에 너무 심취ㅋ ex)한국사시험 p84쪽 두번째 줄에 있던 내용에 집중하는 고런 너낌)
### randomforest -> 
### 장점 : DecisionTree의 오버피팅을 해결하고
### 단점 : 설명력이 좀 어렵다~

In [40]:
from sklearn.externals.six import StringIO

In [41]:
from IPython.display import Image

In [42]:
from sklearn.tree import export_graphviz

In [43]:
import pydotplus

In [44]:
dot_data = StringIO()
export_graphviz(model, out_file=dot_data, rounded=True, special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [192]:
features

['HCLUS', 'PRO_PERCENT', 'HOLIDAY_NEW', 'PROMOTION_NEW']

In [194]:
userInputHCLUS = 1
userInputProPercent = 0.5
userInputHoilidayYn = 1  # "Y"
userInputPromotionYn = 1 # "Y"

In [ ]:
futereData=pd.DataFrame( [userInputHCLUS,userInputProPercent,userInputHoilidayYn, userInputPromotionYn])

In [ ]:
# pickle파일로 저장 가능하다
model_dt.predict()

In [ ]:
import pickle

In [ ]:
filename = 'fianlized_model.sav'
pickle.dump(model_dt, open(filename, 'wb')) #학습용 프로그램은 종료됨


In [ ]:
## 참고
## 머신러닝/딥러닝!!
## lable 인코더 -> 문자 -> 숫자로 맵핑시켜준다 

In [17]:
holiEn = LabelEncoder()

NameError: name 'LabelEncoder' is not defined

In [18]:
featuresData["HOLIDAY_LABEL_EN"] = holiEn.fit_transform(featuresData.HOLIDAY)

NameError: name 'holiEn' is not defined

In [15]:
featuresData["HOLIDAY_DE"] = \
    holiEn.inverse_transform(featuresData.HOLIDAY_LABEL_EN)

0    PG02
Name: PRODUCTGROUP, dtype: object

In [ ]:

featuresData.loc[featuresData.HOLIDAY != featuresData.HOLIDAY_DE]